In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [2]:
import numpy as np

from initialize import initialize
from genregparam import linparams

In [3]:
capacity = 50
intensity = 1.5
slopes = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [4]:
a, b, c, d = initialize(capacity, intensity, rates_init)
e, f = linparams(capacity, intensity, slopes, rates_init)

IndexError: list index out of range

In [ ]:
f